In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np 

import pvlib
from pvlib import clearsky, atmosphere, solarposition
from pvlib.location import Location
from pvlib.iotools import read_tmy3

import warnings
warnings.filterwarnings("ignore")


# Importing data

In [ ]:
GHI_df = pd.DataFrame()
years = np.linspace(2016,2021,6, dtype= int)
file_path = './GHI_dataset/raw_data/GHI_raw_'

for year in years:
    file_path_temp = file_path + str(year) + '.csv'
    df_temp = pd.read_csv(file_path_temp)
    GHI_df = pd.concat([GHI_df, df_temp], axis = 0)

GHI_df['datetime'] = pd.to_datetime(GHI_df['datetime'], format="%Y-%m-%d %H:%M:%S")

GHI_df.set_index('datetime', inplace = True)

In [ ]:
print("Number of GHI_df measurements: " + str(GHI_df.shape[0]))

print("Number of NA: " + str(GHI_df['GHI'].isnull().sum()))

GHI_df.head(10)

## Step 1 - Removal of missing values

In [ ]:
# Define a threshold for the maximum number of consecutive missing values allowed
threshold = 1

# Identify and remove high-density consecutive missing values
def remove_consecutive_nan(df, column, threshold):
    bool_series = df[column].isnull()
    df['block'] = (bool_series.diff(1) != 0).astype('int').cumsum()
    df = df[~((df[column].isnull()) & (df.groupby('block')['block'].transform('size') > threshold))]
    df = df.drop('block', axis=1)
    return df

GHI_df = remove_consecutive_nan(GHI_df, 'GHI', threshold)

print('Number of GHI measurements to interpolate: ' + str(GHI_df['GHI'].isnull().sum()))

In [ ]:
GHI_df['GHI'] = GHI_df['GHI'].interpolate()

print('Number of NaNs: ' + str(GHI_df['GHI'].isnull().sum()))
print("Number of GHI measurements: " + str(GHI_df.shape[0]))

## Step 2 - Identification and removal of outliers

In [ ]:
## Removal of outliers
GHI_max = 1000
GHI_min = 0

GHI_df = GHI_df[(GHI_df["GHI"] < GHI_max) & (GHI_df["GHI"] > GHI_min)]
print("Number of GHI measurements: " + str(GHI_df.shape[0]))

## Step 3 - Clear sky global horizontal irradiance (GHIcs)

In [ ]:
#Clear sky GHI calculation

latitude = 46.518
longitude = 6.565
time_zone = 'Europe/Zurich'
altitude = 400
place = 'Ecublens'
frequency = '10S'

tus = Location(latitude, longitude, time_zone, altitude, place)

cs = tus.get_clearsky(GHI_df.index)
GHI_df['GHIcs'] = cs.ghi


## Step 4 - Removal of night measurements

In [ ]:
# Night is assumed when GHIcs is inferior to 30 W/m2

nb_night_measurements = int(100*GHI_df[GHI_df['GHIcs']<30].shape[0] / GHI_df.shape[0])

GHI_df = GHI_df[GHI_df["GHIcs"] > 30]

print("Percentage of GHI night measurements: " + str(nb_night_measurements) + "%")
print("Number of GHI measurements: " + str(GHI_df.shape[0]))

## Step 5 - Clear sky index (kcs)

In [ ]:
#Calculating the clear sky index - k

GHI_df['k'] = GHI_df["GHI"] / GHI_df["GHIcs"]

#Night measurement have GHIcs = 0 => k=inf ; when it happens, we set k to 0
GHI_df.replace([np.inf, -np.inf], 0, inplace= True)

GHI_df.head(5)

## Step 6 - Calculation of finite-difference

In [ ]:
def FirstOrderBackdDiff (serie, h):
    y_t0 = serie.values.flatten()[1:]
    y_t1 = serie.values.flatten()[0:-1]
    return np.concatenate([[np.nan], (y_t0-y_t1)/(h)])

def SecondOrderBackDiff (serie, h):
    y_t0 = serie.values.flatten()[2:]
    y_t1 = serie.values.flatten()[1:-1]
    y_t2 = serie.values.flatten()[:-2]
    return np.concatenate([[np.nan], [np.nan], (y_t0-2*y_t1+y_t2)/((h)**2)])
    
def ThirdOrderBackDiff(serie, h):
    y_t0 = serie.values.flatten()[3:]
    y_t1 = serie.values.flatten()[1:-2]
    y_t2 = serie.values.flatten()[2:-1]
    y_t3 = serie.values.flatten()[3:]
    return np.concatenate([[np.nan], [np.nan], [np.nan], (y_t0-3*y_t1+3*y_t2-y_t3)/(h**3)])

h = 10 #10s

GHI_df["GHI_d1"] = FirstOrderBackdDiff(GHI_df['GHI'], h)
GHI_df["GHIcs_d1"] = FirstOrderBackdDiff(GHI_df['GHIcs'], h)
GHI_df["k_d1"] = FirstOrderBackdDiff(GHI_df['k'], h)

GHI_df["GHI_d2"] = SecondOrderBackDiff(GHI_df['GHI'], h)
GHI_df["GHIcs_d2"] = SecondOrderBackDiff(GHI_df['GHIcs'], h)
GHI_df["k_d2"] = SecondOrderBackDiff(GHI_df['k'], h)

GHI_df["GHI_d3"] = ThirdOrderBackDiff(GHI_df['GHI'], h)
GHI_df["GHIcs_d3"] = ThirdOrderBackDiff(GHI_df['GHIcs'], h)
GHI_df["k_d3"] = ThirdOrderBackDiff(GHI_df['k'], h)

GHI_df = GHI_df.dropna()

In [ ]:
GHI_df.head(5)

## Step 7 - Adding seasonality

In [ ]:
def norm_sin_hour(x):
    return np.sin(x*np.pi/24)

def norm_sin_day(x):
    return np.sin(x*np.pi/365)

def norm_sin_month(x):
    return np.sin(x*np.pi/12)


GHI_df["month"] = norm_sin_month(GHI_df.index.month).copy()

GHI_df["day"] = norm_sin_day(GHI_df.index.dayofyear).copy()

GHI_df["hour"] = norm_sin_hour(GHI_df.index.hour).copy()

GHI_df.head(5)

## Step 8 - Sampling

In [ ]:
import os
sampling_frequencies = ["15T", "30T", "45T", "1H", "2H", 
                        "4H", "6H", "12H", "24H", "48H", 
                        "72H", "4D", "5D", "6D", "7D"]

sampling_frequencies_eng = ["15_minutes", "30_minutes", "45_minutes", "1_hour", "2_hours", 
                            "4_hours", "6_hours", "12_hours", "24_hours", "48_hours", 
                            "72_hours", "4_days", "5_days", "6_days", "7_days"]

root = "./GHI_dataset/cleaned_sampled_data/"

for i, frequency in enumerate(sampling_frequencies):

    
    df_sampled_temp = GHI_df.resample(frequency).mean().copy()
    df_sampled_temp = df_sampled_temp.dropna()
    df_sampled_temp.reset_index(inplace = True)

    df_sampled_temp['year'] = df_sampled_temp['datetime'].dt.year
    years = df_sampled_temp['year'].unique()

    file_path= root + '/GHI_sampled_' + str(sampling_frequencies_eng[i]) + '.csv'
    df_sampled_temp.to_csv(file_path, index=False)
    